# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data bestaat uit de volgende elementen:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is extracted en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd worden uitgevoerd door het shell command 'Make Data'.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset
from src.features import build_features

# Bond data

In [ ]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

In [ ]:
df_bonds.info()

In [ ]:
df_bonds.head()

Imputeren ontbrekende waarden voor issue rating. 
Issue rating wordt waar deze ontbreekt ingevuld met de meest voorkomende issue rating voor de issuer.

CFI code wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX.

Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd.
Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum.
De reden hiervoor is dat we zo zonder veel moeite de looptijd van de bond kunnen berekenen.

In [ ]:
df_bonds = make_dataset.impute_bonds(df_bonds)

In [ ]:
make_dataset.save_pkl('bonds', df_bonds)

In [ ]:
df_bonds.count()

# Bondprijzen

In [ ]:
df_price = make_dataset.get_price()

In [ ]:
df_price = make_dataset.impute_price(df_price)

In [ ]:
df_price.head()

In [ ]:
df_price.describe()

In [ ]:
df_price.info()

In [ ]:
make_dataset.save_pkl('price', df_price)

# Government Yield curves

In [ ]:
df_yield = make_dataset.get_yield()

In [ ]:
df_yield = make_dataset.impute_yield(df_yield)

In [ ]:
df_yield.tail()

In [ ]:
df_yield.info()

In [ ]:
make_dataset.save_pkl('yield', df_yield)

# Inflation data


In [ ]:
df_inflation = make_dataset.get_inflation()  

In [ ]:
df_inflation = make_dataset.impute_inflation(df_inflation)


df_inflation.info()

In [ ]:
df_inflation.head()

In [ ]:
make_dataset.save_pkl('inflation', df_inflation)

In [ ]:
df_price

In [ ]:
df_bp = make_dataset.join_price(df_bonds,df_price )
df_bp = build_features.add_duration(df_bp)

Om de termspread toe te kunnen voegen moeten we de bond data joinen met de government yield. Hierdoor hebben we minder data beschikbaar.
De vraag is of dit nodig is (of dat het model dit zelf uit kan vogelen)

Hoe meer data we joinen - hoe meer data we kwijt raken. 

In [ ]:
df_yield

In [ ]:
df_bpy = make_dataset.join_yield(df_bp, df_yield)
df_bpy = build_features.add_term_spread(df_bpy)
df_bpy = build_features.add_bid_offer_spread(df_bpy)
df_bpy.info()

In [2]:
df_tf = make_dataset.build_simple_input(df_bonds, df_price)


NameError: name 'make_dataset' is not defined

All in one make statement...

In [4]:
make_dataset.make_data()

2022-01-04 20:13:55.294 | INFO     | src.data.make_dataset:get_bond_data:41 - Load bond data
2022-01-04 20:13:55.296 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\bonds.csv
2022-01-04 20:13:55.391 | INFO     | src.data.make_dataset:impute_bonds:76 - Impute bond data
2022-01-04 20:13:55.400 | INFO     | src.data.make_dataset:save_pkl:375 - Save preprocessed bonds data
2022-01-04 20:13:55.418 | INFO     | src.data.make_dataset:get_price:109 - Load bond price data
2022-01-04 20:13:55.420 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\price.csv
2022-01-04 20:13:55.936 | INFO     | src.data.make_dataset:impute_price:134 - Impute bond price
2022-01-04 20:13:55.956 | INFO     | src.data.make_dataset:save_pkl:375 - Save preprocessed price data
2022-01-04 20:13:59.209 | INFO     | src.data.make_dataset:get_yield:142 - Load goverment yield curve data
2022-01-04 20:13:59.210 | INFO     | src.data.make_dataset:read_csv:26 - Loading d